In [1]:
import requests
import pandas as pd
from io import BytesIO
import os
from io import StringIO

In [2]:
# https://www.dataestur.es/en/apidata/

In [4]:
# Añadimos las urls de las que necesitamos descargar archivos. Key = Name; Value = URL

downloads = {
    "FRONTUR_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/FRONTUR_DL?desde%20%28a%C3%B1o%29=2018&Pa%C3%ADs%20de%20residencia=Todos&Tipo%20de%20visitante=Todos&CCAA%20de%20destino=Todos",
    "TURISMO_INTERNO_PROV_CCAA_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/TURISMO_INTERNO_PROV_CCAA_DL?CCAA%20de%20residencia=Todos&CCAA%20de%20destino=Todos",
    "GASTO_TPV_DESTINO_CCAA_MES_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/GASTO_TPV_DESTINO_CCAA_MES_DL?desde%20(a%C3%B1o)=2018",
    "CST_GASTO_CONSUMO_TURISTICO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/CST_GASTO_CONSUMO_TURISTICO_DL?Producto=Todos"}
   # "EGATUR_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/EGATUR_DL?desde%20%28a%C3%B1o%29=2018&Pa%C3%ADs%20de%20residencia=Todos&CCAA%20de%20destino=Todos",
   # "ACTIVIDADES_OCIO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/ACTIVIDADES_OCIO_DL?CCAA=Todos&Provincia=Todos",
   # "TRANSPORTE_TERRESTRE_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/TRANSPORTE_TERRESTRE_DL?desde%20%28a%C3%B1o%29=2018&Tipo%20transporte=Todos",
    "CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL?Pa%C3%ADs%20origen=Todos&Ciudad%20destino=Todos&Tipo%20origen=Todos",
    "ESCUCHA_ACTIVA_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/ESCUCHA_ACTIVA_DL?Idioma=Todos&Medio=Todos&Sentimiento=Todos",
    "CST_GASTO_CONSUMO_TURISTICO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/CST_GASTO_CONSUMO_TURISTICO_DL?Producto=Todos",
    "ACTIVIDADES_OCIO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/ACTIVIDADES_OCIO_DL?CCAA=Todos&Provincia=Todos"
}

In [5]:
# Creamos una función para descargar un diccionario de URLS. Para ello tenemos que pasarle la url y un nombre de archivo.
# La descarga devuelve un tipo binario, así que usamos BytesIO y lo convertimos en Dataframe con Pandas.

def download_excel(url, file_name, folder="./data"):
    file_path = os.path.join(folder, f"{file_name}.xlsx")

    if os.path.exists(file_path):
        print(f"- {file_name}.xlsx ya existe.")
        return pd.read_excel(file_path)

    response = requests.get(url)
    
    if response.status_code == 200:
        file_type = response.headers["Content-Disposition"].split(".")[-1]

        if file_type == "xlsx":
            df = pd.read_excel(BytesIO(response.content))
            df.to_excel(file_path)
            print(f"{file_name}.xlsx descargado y guardado.")
            return df
        elif file_type == "csv":
            encoding = response.apparent_encoding
            df = pd.read_csv(StringIO(response.content.decode(encoding)), sep=';', engine='python')
            df.to_excel(file_path)
            print(f"{file_name}.csv descargado y guarado.")
            return df
    else:
        print(f"Error al descargar {file_name}: {response.status_code}")
        return None

In [6]:
# Guardamos todos los dataframes. Llamamos nuestra función para crear un nuevo diccionario de dataframes.

datasets = {}

for name, url in downloads.items():
    df = download_excel(url, name)
    if df is not None:
        datasets[name] = df

- CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL.xlsx already exists.
- ESCUCHA_ACTIVA_DL.xlsx already exists.
- CST_GASTO_CONSUMO_TURISTICO_DL.xlsx already exists.
- ACTIVIDADES_OCIO_DL.xlsx already exists.


In [223]:
#Para visualizar datos en conjunto:

for name, df in datasets.items():
    print(f"\n {name}")
    print("Columns:", df.columns.tolist())
    print("First 3 rows:\n", df.head(1))


 FRONTUR_DL
Columns: ['Unnamed: 0', 'AÑO', 'MES', 'PAIS_RESIDENCIA', 'TIPO_VISITANTE', 'CCAA_DESTINO', 'VISITANTES']
First 3 rows:
    Unnamed: 0   AÑO  MES PAIS_RESIDENCIA TIPO_VISITANTE CCAA_DESTINO  \
0           0  2018    1        Alemania  Excursionista    Andalucía   

   VISITANTES  
0         818  

 TURISMO_INTERNO_PROV_CCAA_DL
Columns: ['Unnamed: 0', 'AÑO', 'MES', 'CCAA_ORIGEN', 'PROVINCIA_ORIGEN', 'CCAA_DESTINO', 'PROVINCIA_DESTINO', 'TURISTAS', 'PERNOCTACIONES', 'ESTANCIA_MEDIA']
First 3 rows:
    Unnamed: 0   AÑO  MES CCAA_ORIGEN PROVINCIA_ORIGEN CCAA_DESTINO  \
0           0  2019    7   Andalucía          Almería    Andalucía   

  PROVINCIA_DESTINO TURISTAS PERNOCTACIONES ESTANCIA_MEDIA  
0             Cádiz   8910,0        33536,0            3,8  

 GASTO_TPV_DESTINO_CCAA_MES_DL
Columns: ['Unnamed: 0', 'AÑO', 'MES', 'CCAA_DESTINO', 'TIPO_ORIGEN', 'GASTO', 'TRANSACCION']
First 3 rows:
    Unnamed: 0   AÑO  MES CCAA_DESTINO    TIPO_ORIGEN        GASTO  TRANSACCION
0   

In [224]:
datasets["FRONTUR_DL"].head(2)

,Unnamed: 0,AÑO,MES,PAIS_RESIDENCIA,TIPO_VISITANTE,CCAA_DESTINO,VISITANTES
0,0,2018,1,Alemania,Excursionista,Andalucía,818
1,1,2018,1,Alemania,Excursionista,Baleares,3694


In [209]:
datasets["TURISMO_INTERNO_PROV_CCAA_DL"].head(2)

,Unnamed: 0,AÑO,MES,CCAA_ORIGEN,PROVINCIA_ORIGEN,CCAA_DESTINO,PROVINCIA_DESTINO,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA
0,0,2019,7,Andalucía,Almería,Andalucía,Cádiz,"8910,0","33536,0","3,8"
1,1,2019,7,Andalucía,Almería,Andalucía,Córdoba,"3468,0","11981,0","3,5"


In [210]:
datasets["GASTO_TPV_DESTINO_CCAA_MES_DL"].head(2)

,Unnamed: 0,AÑO,MES,CCAA_DESTINO,TIPO_ORIGEN,GASTO,TRANSACCION
0,0,2019,1,Andalucía,Internacional,"18481196,78",414192
1,1,2019,1,Andalucía,Interregional,"34039254,92",952493


In [248]:
datasets["CST_GASTO_CONSUMO_TURISTICO_DL"].head(2)

,Unnamed: 0,AÑO,PRODUCTO,NUM_GASTO_INTERIOR,NUM_GASTO_INTERNO,NUM_GASTO_EMISOR,NUM_CONSUMO_TURISTICO_INTERIOR,NUM_CONSUMO_OTROS_COMPONENTES_TURISTICOS,NUM_GASTO_RECEPTOR
0,0,2016,Agencias de viajes y otros servicios de reserva,6910.6,5090.6,3509.2,7021.3,110.7,1819.9
1,1,2016,Hoteles y similares,23393.6,9216.4,4030.7,23467.8,74.2,14177.2


In [211]:
df_internacional = datasets["FRONTUR_DL"].copy()
df_internacional.groupby("CCAA_DESTINO")["VISITANTES"].sum().sort_values(ascending=False).head()

CCAA_DESTINO
Cataluña      161578411
Canarias       88495347
Andalucía      88145477
Baleares       85091689
Transito       80351513
Name: VISITANTES, dtype: int64

In [253]:
df_nacional = datasets["TURISMO_INTERNO_PROV_CCAA_DL"].copy()
df_nacional.groupby("CCAA_DESTINO")["TURISTAS"].sum().sort_values(ascending=False).head()

CCAA_DESTINO
Andalucía               370270805
Cataluña                289584593
Comunidad de Madrid     199334614
Comunitat Valenciana    194219610
Castilla y León         191184938
Name: TURISTAS, dtype: int32

In [260]:
print(df_gasto["TIPO_ORIGEN"].unique())

['Internacional' 'Interregional' 'Regional' 'Total Nacional']


In [258]:
df_gasto = datasets["GASTO_TPV_DESTINO_CCAA_MES_DL"].copy()
df_gasto[(df_gasto["TIPO_ORIGEN"] == "Internacional")].groupby("CCAA_DESTINO")["GASTO"].sum().sort_values(ascending=False).head()

CCAA_DESTINO
La Rioja           97428,4576797,61111560,07167017,83330140,28247...
Castilla y León    793046,86568247,28792382,971096406,491282510,1...
Melilla            59660,9431020,6148212,461709,8955731,8264523,8...
País Vasco         5570422,895433257,626641640,99378513,571170674...
Extremadura        527429,96512252,98605432,8729106,79685418,6969...
Name: GASTO, dtype: object

In [265]:
df_gasto = datasets["GASTO_TPV_DESTINO_CCAA_MES_DL"].copy()

df_gasto[df_gasto["TIPO_ORIGEN"].isin(["Regional", "Interregional"])].groupby("CCAA_DESTINO")["GASTO"].sum().sort_values(ascending=False).head()

CCAA_DESTINO
País Vasco    8841938,644143767,227574824,083874917,01894467...
Ceuta         530914,92424565,29489429,22502344,43522909,185...
Galicia       5087195,754841505,984176236,374192668,11520380...
La Rioja      4207155,633689597,134788193,045646755,55482550...
Cantabria     3863833,373762142,094902663,636631977,96535474...
Name: GASTO, dtype: object

In [256]:
df_gasto.groupby("TIPO_ORIGEN")["GASTO"].sum()

TIPO_ORIGEN
Internacional     18481196,78503190,6821139385,8251236,14293597,...
Interregional     34039254,9210511516,0713773987,423863833,37172...
Regional          33721507,961976147,615307096,342904776,6367096...
Total Nacional    67760762,8812487663,6819081083,763863833,37201...
Name: GASTO, dtype: object

In [7]:
# ACTIVIDADES DE OCIO - RAMA FLOR
# Creamos un display del DataFrame de las actividades de ocio en su integralidad
df_AOD = datasets["ACTIVIDADES_OCIO_DL"]
df_AOD


,AÑO,MES,PRODUCTO,CATEGORIA,CCAA,PROVINCIA,ENTRADAS,VISITAS_PAGINAS,GASTO_TOTAL,PRECIO_MEDIO_ENTRADA,TRANSACCIONES
0,2023,7,Espectaculos,"Cultura, teatro y danza",País Vasco,Araba/Álava,35,135,420.0000,12.0000,12
1,2023,7,Espectaculos,"Cultura, teatro y danza",País Vasco,Araba/Álava,2,8,6.4600,3.2300,1
2,2023,7,Espectaculos,Música,País Vasco,Araba/Álava,310,4171,3357.3000,10.8300,153
3,2023,7,Actividades,Actividades infantiles,País Vasco,Araba/Álava,39,151,207.8700,5.3300,17
4,2023,7,Espectaculos,"Cultura, teatro y danza",País Vasco,Araba/Álava,290,1123,2726.0000,9.4000,124
...,...,...,...,...,...,...,...,...,...,...,...
17207,2025,6,Actividades,Deportes y aventuras,Total Nacional,Total Nacional,3505,10282,99371.3065,28.3513,2250
17208,2025,6,Actividades,Actividades infantiles,Total Nacional,Total Nacional,10184,28670,189904.1032,18.6473,3980
17209,2025,6,Espectaculos,"Cultura, teatro y danza",Total Nacional,Total Nacional,38317,103158,550569.3096,14.3688,17599
17210,2025,6,Espectaculos,Museo y exposiciones,Total Nacional,Total Nacional,1086,3291,13221.1812,12.1742,324


In [8]:
# En las categorias, creamos un diccionario para tener las diferentes categorias de atracciones en funcion del producto 
# Observamos que la Música, La cultura, teatro y danza y las actividades Infantiles son las tres attracciones principales. 
df_AOD[["PRODUCTO","CATEGORIA"]].value_counts().to_dict()

{('Espectaculos', 'Música'): 7404,
 ('Espectaculos', 'Cultura, teatro y danza'): 3454,
 ('Actividades', 'Actividades infantiles'): 2118,
 ('Espectaculos', 'Musicales'): 883,
 ('Actividades', 'Deportes y aventuras'): 845,
 ('Actividades', 'Cursos'): 715,
 ('Actividades', 'Gastronomía'): 585,
 ('Espectaculos', 'Cine'): 485,
 ('Espectaculos', 'Museo y exposiciones'): 303,
 ('Espectaculos', 'Conferencias'): 284,
 ('Espectaculos', 'Ferias'): 116,
 ('Espectaculos', 'Parques temáticos'): 20}

In [9]:
# Dividimos las diferentes categorias de actividades por años en funcion del numero total de entradas.
categoria_grupos = df_AOD.groupby(["AÑO", "CATEGORIA"])["ENTRADAS"].sum().unstack()
categoria_grupos

CATEGORIA,Actividades infantiles,Cine,Conferencias,"Cultura, teatro y danza",Cursos,Deportes y aventuras,Ferias,Gastronomía,Museo y exposiciones,Musicales,Música,Parques temáticos
AÑO,,,,,,,,,,,,
2023,230456.0,18118.0,5246.0,489258.0,9612.0,47484.0,2592.0,23104.0,25356.0,83566.0,1575924.0,1276.0
2024,500268.0,46034.0,36966.0,1097994.0,32338.0,119118.0,6524.0,47774.0,47668.0,180822.0,3415368.0,2214.0
2025,218646.0,24496.0,9382.0,564554.0,22082.0,48796.0,4472.0,26324.0,18990.0,95536.0,1743530.0,NaN


In [11]:
# Definimos las diferentes categorias segun los años 2023, 2024 Y 2025. 
df_AOD.groupby(["AÑO", "CATEGORIA"])["ENTRADAS"].sum().reset_index().sort_values("ENTRADAS", ascending=False)

,AÑO,CATEGORIA,ENTRADAS
22,2024,Música,3415368
34,2025,Música,1743530
10,2023,Música,1575924
15,2024,"Cultura, teatro y danza",1097994
27,2025,"Cultura, teatro y danza",564554
12,2024,Actividades infantiles,500268
3,2023,"Cultura, teatro y danza",489258
0,2023,Actividades infantiles,230456
24,2025,Actividades infantiles,218646
21,2024,Musicales,180822


In [12]:
# Numero total de entradas segun las categorias. 
categoria_grupos = df.groupby(["CATEGORIA"])["ENTRADAS"].sum().sort_values(ascending = False)
categoria_grupos

CATEGORIA
Música                     6734822
Cultura, teatro y danza    2151806
Actividades infantiles      949370
Musicales                   359924
Deportes y aventuras        215398
Gastronomía                  97202
Museo y exposiciones         92014
Cine                         88648
Cursos                       64032
Conferencias                 51594
Ferias                       13588
Parques temáticos             3490
Name: ENTRADAS, dtype: int64

In [38]:
# Cogemos la categoria musica, y seleccionamos las 3 communidades que mas entradas hacen gracias a los eventos de musica. 
df[(df["CATEGORIA"] == "Música") & (df["CCAA"] != "Total Nacional")].groupby("CCAA")["ENTRADAS"].sum().sort_values(ascending = False).head(3)

CCAA
Comunidad de Madrid    760694
Cataluña               696130
Illes Balears          488764
Name: ENTRADAS, dtype: int64

In [42]:
# Cogemos la categoria Cultura, teatro y danza, y seleccionamos las 3 communidades que mas entradas hacen gracias a los eventos de Cultura, teatro y danza. 
df[(df["CATEGORIA"] == "Cultura, teatro y danza") & (df["CCAA"] != "Total Nacional")].groupby("CCAA")["ENTRADAS"].sum().sort_values(ascending = False).head(3)

CCAA
Cataluña                176517
Andalucía               144106
Comunitat Valenciana    135913
Name: ENTRADAS, dtype: int64

In [40]:
# Cogemos la categoria Actividades infantiles, y seleccionamos las 3 communidades que mas entradas hacen gracias a los eventos de Actividades infantiles. 
df[(df["CATEGORIA"] == "Actividades infantiles") & (df["CCAA"] != "Total Nacional")].groupby("CCAA")["ENTRADAS"].sum().sort_values(ascending = False).head(3)

CCAA
Cataluña                104608
Comunitat Valenciana     87822
Andalucía                46486
Name: ENTRADAS, dtype: int64

In [16]:
# creamos una tabla con las categorias de ocios en funcion de las comunidades españolas. 
tabla = df_AOD.groupby(["CATEGORIA", "CCAA"])["ENTRADAS"].sum().unstack(fill_value=0)

totales = tabla.sum(axis=1)
tabla_orden = tabla.loc[totales.sort_values(ascending=False).index]
display(tabla_orden)

CCAA,Andalucía,Aragón,Canarias,Cantabria,Castilla - La Mancha,Castilla y León,Cataluña,Ceuta,Comunidad Foral de Navarra,Comunidad de Madrid,Comunitat Valenciana,Extremadura,Galicia,Illes Balears,La Rioja,País Vasco,Principado de Asturias,Región de Murcia,Total Nacional
CATEGORIA,,,,,,,,,,,,,,,,,,,
Música,337814,131441,172083,11558,39082,144733,696130,761,85976,760694,195139,15383,83488,488764,11410,103304,35372,54279,3367411
"Cultura, teatro y danza",144106,53581,107435,7211,50571,77233,176517,695,1823,120586,135913,13681,19585,77271,9307,63538,2671,14179,1075903
Actividades infantiles,46486,24819,13743,3730,14451,33291,104608,0,9196,31192,87822,6454,6791,40752,1431,43363,1981,4575,474685
Musicales,17284,10264,12297,387,16381,7784,33511,0,124,22052,36880,10144,2462,6167,243,294,430,3258,179962
Deportes y aventuras,10532,3843,7357,0,2314,2270,22886,0,5045,13058,5007,7016,1129,3823,448,20245,1040,1686,107699
Gastronomía,7208,1092,2588,693,1446,1047,16131,0,82,5852,4483,116,134,5738,398,650,708,235,48601
Museo y exposiciones,12427,2252,23,0,2768,7886,3037,0,0,41,451,0,16,360,0,16734,12,0,46007
Cine,10200,3457,671,4,2464,566,8283,0,0,3009,6252,0,184,1696,26,5442,6,2064,44324
Cursos,2995,1124,1268,24,212,189,8374,0,8,6941,801,208,159,6363,38,978,13,2321,32016


In [19]:
df["CCAA"].value_counts()

CCAA
Cataluña                      3446
Andalucía                     2276
Illes Balears                 1781
Comunitat Valenciana          1430
Canarias                      1161
Comunidad de Madrid           1109
País Vasco                    1047
Castilla y León               1041
Castilla - La Mancha           760
Galicia                        629
Extremadura                    562
Aragón                         556
Región de Murcia               313
Total Nacional                 272
Principado de Asturias         264
Comunidad Foral de Navarra     227
La Rioja                       165
Cantabria                      147
Ceuta                           26
Name: count, dtype: int64